## Problema rellenar una mochila sin valores

In [2]:
import numpy as np

In [3]:
def tensor_generator(pesos:np.array, n_elementos:np.array, capacidad:int, tao:float):
    lista_de_tensores = []
    n_clases = len(pesos)
    tensor = tensor_initial_generator(pesos[0],n_elementos[0],capacidad,0)

    lista_de_tensores.append(tensor)

    for n in range(1, n_clases-1):
        tensor = tensor_intermediate_generator(pesos[n], n_elementos[n], capacidad, tensor.shape[1])
        lista_de_tensores.append(tensor)

    tensor = tensor_final_generator(pesos[-1], n_elementos[-1], capacidad, tensor.shape[1],tao)
    lista_de_tensores.append(tensor)

    return lista_de_tensores



In [4]:
def tensor_initial_generator(peso:int, n_elementos:int, capacidad:int, previous_weight_solution:int =0)->np.array:
    """
    Funcion que genera el tensor inicial.
    """
    tamaño_1 = min((capacidad-previous_weight_solution)//peso,n_elementos)+1
    tamaño_2 = min(capacidad, (tamaño_1-1)*peso+previous_weight_solution)+1
    tensor = np.zeros((tamaño_1,tamaño_2))
    for i in range(tamaño_1):
        elemento = i*peso + previous_weight_solution
        tensor[i][elemento] = 1
    return tensor

In [5]:
def tensor_intermediate_generator(peso:int, n_elementos:int, capacidad:int, previous_weight:int)->np.array:

    tamaño_1 = previous_weight
    tamaño_2 = min(capacidad, n_elementos*peso+previous_weight-1)+1
    tensor = np.zeros((tamaño_1,tamaño_2))
    for i in range(tamaño_1):
        n_elementos_restantes = min((capacidad -i)//peso,n_elementos)+1
        for  j in range(n_elementos_restantes):
            elemento = i + j*peso
            tensor[i][elemento] = 1

    return tensor

In [6]:
def tensor_final_generator(peso:int, n_elementos: int, capacidad: int, previous_weight:int, tao:float)->np.array:
    tamaño_1 = previous_weight
    tensor = np.zeros((tamaño_1))
    for i in range(tamaño_1):
        n_elementos_posibles = min((capacidad -i)//peso,n_elementos)+1
        for j in range(n_elementos_posibles):
            elemento = i +j*peso
            tensor[i] += np.exp(tao*elemento)
            #tensor[i] += np.exp(-tao*(capacidad - elemento))

    return tensor

In [7]:
def tensor_contraction(lista_de_tensores:list):

    n_tensores = len(lista_de_tensores)
    tensores_intermedios = []
    vector = lista_de_tensores[-1]
    tensores_intermedios.append(vector)
    for j in range(n_tensores-1,0,-1):
        vector = lista_de_tensores[j-1] @ vector

        tensores_intermedios.append(vector)
    tensores_intermedios.reverse()
    return vector,tensores_intermedios


In [8]:
def solver(pesos:np.array, n_elementos:np.array, capacidad:int, tao:float)->np.array:
    """
    Funcion que resuelve el problema de la mochila sin valores.
    """ 
    n_clases = len(pesos)
    solution = np.zeros(n_clases)
    lista_de_tensores = tensor_generator(pesos,n_elementos, capacidad, tao)

    vector_salida, tensores_intermedios = tensor_contraction(lista_de_tensores)

    solution[0] = np.argmax(vector_salida)


    for n in range(1,n_clases-1):
        cuenta_peso = int(np.dot(solution[:n],pesos[:n]))
        new_initial_tensor = tensor_initial_generator(pesos[n], n_elementos[n], capacidad, cuenta_peso)
        solution[n]= np.argmax(new_initial_tensor @ tensores_intermedios[n+1][:new_initial_tensor.shape[1]])

    cuenta_peso = np.dot(solution[:-1],pesos[:-1])
    solution[-1] = min((capacidad-cuenta_peso)//pesos[-1],n_elementos[-1])
    #print("La solucion es: ", solution)
    #print("El peso total es: ", np.dot(solution,pesos))
    return solution

In [9]:
clases = 50
capacidad = 30000
np.random.seed(1)
pesos = np.random.randint(1,10,clases)
n_elementos = [3]*clases

tao = 0.2

b=solver(pesos, n_elementos, capacidad, tao)


In [13]:
import matplotlib.pyplot as plt
from time import time
tao=0.1
num_clases=np.arange(10,100)
tiempo=[]
precision=[]
for j in num_clases:
    pesos = np.random.randint(1,10,j)
    n_elementos = [3]*j
    capacidad = j*10
    inicio=time()
    b=solver(pesos, n_elementos, capacidad, tao)
    resultado=np.dot(b,pesos)
    fin=time()
    tiempo.append(fin-inicio)
    precision.append(resultado/capacidad)



plt.plot(num_clases, precision, marker='o', color='b', linestyle='-')  # 'o' para puntos, 'b' para color azul, '-' para línea sólida

# Añadir etiquetas y título
plt.xlabel("num_clases")
plt.ylabel("tiempo")
plt.title("Gráfico de línea con dos listas")

# Mostrar el gráfico
plt.show()

NameError: name 'np' is not defined